## 注意(attention)！开始做前必读项！
所有的代码一定要在这个文件里面编写，不要自己创建一个新的文件 对于提供的数据集，不要改存储地方，也不要修改文件名和内容 不要重新定义函数（如果我们已经定义好的），按照里面的思路来编写。当然，除了我们定义的部分，如有需要可以自行定义函数或者模块 写完之后，重新看一下哪一部分比较慢，然后试图去优化。一个好的习惯是每写一部分就思考这部分代码的时间复杂度和空间复杂度，AI工程是的日常习惯！ 
这次作业很重要，一定要完成！ 相信会有很多的收获！

In [2]:
import numpy as np
import pandas as pd
from gensim import models
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

## 加载训练集文件和测试集文件

In [13]:
data_text,data=[],[]
stopWords=[]

def load_data():
    '''
    函数说明：该函数用于加载数据集
    return: 
        -data: 表示所有数据拼接的原始数据
        -data_text: 表示数据集中的特征数据集
        -datatext: 表示经过分词之后的特征数据集
        -stopWords: 表示读取的停用词
    '''
    print('load Pre_process')
    data = pd.concat([
        pd.read_csv('train_clean.csv', sep='\t'),
        pd.read_csv('dev_clean.csv', sep='\t'),
        pd.read_csv('test_clean.csv', sep='\t')
        ])
    print("读取数据集完成")
    data_text = list(data.text)  # .apply(lambda x: x.split(' '))
    datatext = []
    for i in range(len(data_text)):
        datatext.append(data_text[i].split(' '))
    stopWords = open('stopwords(1).txt', 'r', encoding='utf-8').readlines()
    print("取停用词完成")
    return data, data_text,datatext, stopWords


In [14]:
data, data_text, datatext, stopWords=load_data()

load Pre_process
读取数据集完成
取停用词完成


In [23]:
def trainer_tfidf(data, stopWords):
    '''
    函数说明：该函数用于训练tfidf的词向量
    return: 
        -tfidf: 表示经过TF-ID模型训练出的词向量
    '''
    # ToDo
    # 根据数据集训练tfidf的词向量
    # 第一步：首先通过TfidfVectorizer创建一个模型对象
    count_vect = TfidfVectorizer(stop_words=stopWords, max_df=0.6, ngram_range=(1,2))
    # 第二步：用模型对象去fit训练数据集
    tfidf = count_vect.fit(data)
    print('train tfidf_embedding')
    #返回是一个稀疏矩阵
    return tfidf


In [25]:
tfidf = trainer_tfidf(data_text, stopWords)

train tfidf_embedding


In [31]:
x = tfidf.transform(data_text)

根据数据集训练word2vec的词向量  
利用models.Word2Vec训练w2v的词向量  
部分参数说明：  
min_count：表示低于该频率的词将会删除，  
window：表示滑动窗口大小，  
alpha：表示学习率，  
negative：表示负采样样本个数，  
max_vocab_size：表示RAM中最大限制的词个数  

In [45]:
def trainer_w2v():
    '''
    函数说明：该函数基于Word2vec模型训练词向量
    return: 
        -w2v: 表示经过word2vec模型训练出的词向量
    '''
    print('train word2vec Embedding')
    # 训练w2v的词向量
    # 第一步：利用models.Word2Vec构建一个模型，
    w2v = models.Word2Vec(min_count=2,
                                window=3,
                                vector_size=300,
                                sample=6e-5,
                                alpha=0.03,
                                min_alpha=0.0007,
                                negative=15,
                                workers=4,
                                max_vocab_size=50000) 
        
    w2v.build_vocab(datatext)

    w2v.train(datatext,
              total_examples=w2v.corpus_count,
              epochs=15,
              report_delay=1)
        
    print('train fast_embedding')
    return w2v

In [46]:
w2v = trainer_w2v()

train word2vec Embedding
train fast_embedding


In [62]:
def trainer_fasttext():
    '''
    函数说明：该函数基于FastText模型训练词向量
    return: 
        -fast: 表示经过FastText模型训练出的词向量
    '''
    # ToDo
    # 根据数据集训练FastTExt的词向量
    # hint: 利用models.FastText,
    # 可使用部分参数说明：
    # vector_size：生成的向量维度，
    # window: 移动窗口，
    # aphla: 学习率，
    # min_count: 对低于该频率的词进行截断
    # 可以参照trainer_w2v函数完成FastText的词向量的训练
    # 可以直接根据models.FastText直接训练样本获取词向量
    fast = models.FastText(vector_size=300,
                           window=3,
                           alpha=0.025,
                           min_count=2)
    fast.build_vocab(datatext)
    fast.train(datatext, total_examples=fast.corpus_count, epochs=10)
    return fast

In [63]:
fast= trainer_fasttext()

In [64]:
def saver():
    '''
    函数说明：该函数存储训练好的模型
    '''
    #Todo
    # hint: 通过joblib.dump保存tfidf
    print('save tfidf model')
    joblib.dump(tfidf, './tfidf_model')
    # hint: w2v可以通过自带的save函数进行保存
    w2v.save('./w2v_model')
    print('save word2vec model')
     # hint: fast可以通过自带的save函数进行保存
    fast.save('./fast_model')
    print('save fast model')

    
saver()
    

save tfidf model
save word2vec model
save fast model


In [65]:
def load():
    '''
    函数说明：该函数加载训练好的模型
    '''
    #ToDo
    # 加载模型 
    # hint: tfidf可以通过joblib.load进行加载
    # w2v和fast可以通过gensim.models.KeyedVectors.load加载
    print('load tfidf_embedding model')
    tfidf = joblib.load('./tfidf_model')
    print('load w2v_embedding model')
    w2v =  models.Word2Vec.load('./w2v_model')
    print('load fast_embedding model')
    fast =  models.FastText('./fast_model')
load()

load tfidf_embedding model
load w2v_embedding model
load fast_embedding model


RuntimeError: you must first build vocabulary before training the model